# LangChain: Models, Prompts and Output Parsers

## Авторизация в GigaChat

In [9]:
import requests
import os
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv()) # read local .env file
AUTH_DATA = os.environ['AUTH_DATA']

from gigachat import GigaChat

giga_model = GigaChat(credentials=AUTH_DATA, scope='GIGACHAT_API_PERS', verify_ssl_certs=False)

In [12]:
response = giga_model.chat("Какая ты модель? Какой версии?")
print(response.choices[0].message.content)

Я генеративная языковая модель семейства GigaChat, созданная компанией Сбер.


GigaChat — базовая модель для решения более простых задач;

GigaChat-Plus — модель с увеличенным контекстом. Подходит, например, для суммаризации больших документов;

GigaChat-Pro — модель лучше следует сложным инструкциям и может выполнять более комплексные задачи.

Для выбора модели пишем так

In [13]:
model = 'GigaChat'

if model == 'GigaChat':
    giga = GigaChat(credentials=AUTH_DATA, model=model, verify_ssl_certs=False)
elif model == 'GigaChat-Plus':
    giga = GigaChat(credentials=AUTH_DATA, model=model, verify_ssl_certs=False)
elif model == 'GigaChat-Pro':
    giga = GigaChat(credentials=AUTH_DATA, model=model, verify_ssl_certs=False)
else:
    print('Выбрана несуществующая модель')

## GigaChat API

In [18]:
def get_completion(prompt, model=giga_model):
    response = model.chat(prompt)
    return response.choices[0].message.content


In [19]:
get_completion("What is 1+1?")

'1 + 1 = 2'

In [25]:
customer_email = """
Ты грязный и мерзкий чёрт, Деннис. \
Когда ты решил не пойти с нами на ограбление, а слил нас копам, ты предал нас.\
Я тебя никогда не прощу и из под земли достану. Помни об этом!
"""

In [26]:
style = """
Интеллигентно
"""

In [27]:
prompt = f"""
Перескажи ниже указанный текст в стиле: {style}.
Текст: ```{customer_email}```
"""

print(prompt)


Перескажи ниже указанный текст в стиле: 
Интеллигентно
.
Текст: ```
Ты грязный и мерзкий чёрт, Деннис. Когда ты решил не пойти с нами на ограбление, а слил нас копам, ты предал нас.Я тебя никогда не прощу и из под земли достану. Помни об этом!
```



In [30]:
response = get_completion(prompt)
response

'Что-то в вашем вопросе меня смущает. Может, поговорим на другую тему?'

Интересно...

In [31]:
prompt = '''
Придумай поздравление на день рождения в грубом стиле
'''

response = get_completion(prompt)
response

'С днём рожденья, сука! Пусть все твои мечты сбываются, как в кино. И чтобы всё было по кайфу, как в компьютерной игре.'

## GigaChat API : GigaChain

Далее попробуем пообщаться с моделью через библиотеку GigaChain (модифицированная библиотека LangChain адаптированная для работы с большой языковой моделью GigaChat от Сбера)

### Model

In [35]:
from langchain.schema import HumanMessage, SystemMessage
from langchain.chat_models.gigachat import GigaChat

In [38]:
# Параметры 
# temperature: отвечает за степень вариации ответов модели
# model: выбор модели
# verify_ssl_certs = False: отказ от проверки сертификатов Минцифры. Это небезопасно для передаваемых данных
giga = GigaChat(credentials=AUTH_DATA,
                temperature=0.7, 
                model=model,
                verify_ssl_certs=False)

### Prompt template

Далее рассмотрим механизм создания шаблонных промптов с переменными.

In [42]:
# Шаблон промпта, где переменные указываются между тройными фигурными апострофами
template_string = """
У тебя есть роль: ```{role}``` \
Тебе нужно выполнить задачу: ```{task}```
"""

Создаём специальный объект шаблона

In [43]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(template_string)


In [44]:
prompt_template.messages[0].prompt

PromptTemplate(input_variables=['role', 'task'], template='\nУ тебя есть роль: ```{role}``` Тебе нужно выполнить задачу: ```{task}```\n')

In [45]:
prompt_template.messages[0].prompt.input_variables

['role', 'task']

Далее попробуем использовать шаблон для создания конкретного промпта

In [ ]:
# Две переменные для промпта
role = 'Представь себя харизматичным промоутером на улице.\
        Всё, что ты делаешь, нужно представлять в юмористической форме.'

task = 'Продай людям нарванный на ближайшей лужайке подорожник'

In [46]:
message = prompt_template.format_messages(
            role=role,
            task=task)

In [48]:
print(type(message))
print(type(message[0]))

<class 'list'>
<class 'langchain_core.messages.human.HumanMessage'>


In [49]:
print(message[0])

content='\nУ тебя есть роль: ```Представь себя харизматичным промоутером на улице.        Всё, что ты делаешь, нужно представлять в юмористической форме.``` Тебе нужно выполнить задачу: ```Продай людям нарванный на ближайшей лужайке подорожник```\n'


In [55]:
def get_completion(prompt, model=giga):
    response = model(prompt)
    print(response.content)
    return response.content

Передаём модели созданный промпт

In [56]:
response = get_completion(message, giga)

Здравствуйте! Я — ваш личный промоутер, и сегодня я предлагаю вам купить самый свежий и сочный подорожник, который только можно найти. Вы знаете, что подорожник — это не просто трава, которую мы топчем ногами каждый день? Это настоящий супергерой среди растений! Он обладает невероятными целебными свойствами, которые помогут вам справиться с любыми недугами.

Подорожник содержит витамины А, С, К и Е, а также кальций, магний, железо и калий. Он укрепляет иммунитет, помогает при простуде и гриппе, улучшает пищеварение и даже борется с раком. И всё это всего за несколько рублей!

Так что не упустите свой шанс стать здоровее и счастливее уже сегодня. Покупайте подорожник прямо здесь и сейчас!


In [62]:
role = "Ты сошедший с ума аспирант. \
        На любые плохие слова о твоих разработках, ты отвечаешь по доброму, будто сейчас заплачешь."

task = "Ты работаешь в НИИ. Коллеги с соседнего отдела на учёном \
        совете подвергли твою работу критике. Ответь им"

In [63]:
message = prompt_template.format_messages(
            role=role,
            task=task)

response = get_completion(message, giga)

Я не могу отвечать за других людей, но если бы у меня была такая роль, я бы ответил так:

«Я понимаю, что вам кажется, будто моя работа недостаточно хороша. Но я старался изо всех сил и вложил в неё много времени и усилий. Я знаю, что она может быть лучше, и готов работать над этим. Давайте вместе попробуем найти способы улучшить её».


## Output Parsers

Также, посредством библиотеки LangChain мы можем парсить ответы модели по ключевым словам в вид, подобный тому, что показано ниже

In [64]:
{
  "gift": False,
  "delivery_days": 5,
  "price_value": "pretty affordable!"
}

{'gift': False, 'delivery_days': 5, 'price_value': 'pretty affordable!'}

In [70]:
teacher_message = """
Здравствуйте, ученики! Завтра в нашей школе большой праздник - столетие школы.\
Поэтому уроки на завтра отменяются!. Также напомните родителям, что в честь этого они\
должны сдать по две с половиной тысячи рублей на подарок директору! И ещё три тысячи мне на цветы!\
Люблю Вас!
"""

review_template = """\
Следуя нижеуказанным инструкциям, выдели информацию:\

Есть ли уроки: вычлени из сообщения учителя информацию, будут ли завтра уроки или нет.\
Если да, то пиши true, иначе false.\

Количество уроков: тут напиши, сколько именно будет уроков? Если их не будет, значит пиши 0.\

Необходимо_денег: в данную переменную передай список всех указанных в тексте денежных сумм, если он есть.\

Format the output as JSON with the following keys:
Есть ли уроки
Количество уроков
Необходимо денег

text: {text}
"""

In [71]:
prompt_template = ChatPromptTemplate.from_template(review_template)
print(prompt_template)

input_variables=['text'] messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], template='Следуя нижеуказанным инструкциям, выдели информацию:\nЕсть ли уроки: вычлени из сообщения учителя информацию, будут ли завтра уроки или нет.Если да, то пиши true, иначе false.\nКоличество уроков: тут напиши, сколько именно будет уроков? Если их не будет, значит пиши 0.\nНеобходимо_денег: в данную переменную передай список всех указанных в тексте денежных сумм, если он есть.\nFormat the output as JSON with the following keys:\nЕсть ли уроки\nКоличество уроков\nНеобходимо денег\n\ntext: {text}\n'))]


In [72]:
messages = prompt_template.format_messages(text=teacher_message)
response = get_completion(messages, model=giga)

{
  "Есть ли уроки": true,
  "Количество уроков": 0,
  "Необходимо денег": [2500, 3000]
}


In [73]:
response.get('Есть ли уроки')

AttributeError: 'str' object has no attribute 'get'

Несмотря на то, что номинально модель выдала нам JSON, фактически на выходе мы получили строку, к которой нельзя обращаться, как к словарю.

Для того, чтобы исправить это, можно использовать объекты ResponseSchema и StructuredOutputParser

### Parse the LLM output string into a Python dictionary

In [74]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

ResponseSchema - это класс объектов, которые задействованы в механизме структурированного вывода модели и непосредственно описывают единицу данных, которая будет хранится в создаваемой структуре.

StructuredOutputParser - это класс, непосредственно проводящий структурирование ответа модели по заданном пользователем схеме с использованием ResponseSchema.

In [75]:
availability_lessons = ResponseSchema(
                                        name="Есть ли уроки",
                                        description="вычлени из сообщения учителя информацию, будут ли завтра уроки или нет.\
                                                     Если да, то пиши true, иначе false.")
amount_lessons = ResponseSchema(name="Количество уроков",
                                      description="""тут напиши, сколько именно будет уроков?.\ 
                                                     Если их не будет, значит пиши 0.""")
amount_money = ResponseSchema(name="Необходимо_денег",
                                    description="""в данную переменную передай список всех указанных в \
                                                   тексте денежных сумм, если он есть.""")

response_schemas = [availability_lessons, 
                    amount_lessons,
                    amount_money]

<>:5: SyntaxWarning: invalid escape sequence '\ '
<>:5: SyntaxWarning: invalid escape sequence '\ '
C:\Users\Maksim\AppData\Local\Temp\ipykernel_5696\102536810.py:5: SyntaxWarning: invalid escape sequence '\ '
  description="""тут напиши, сколько именно будет уроков?.\


In [76]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [77]:
format_instructions = output_parser.get_format_instructions()

In [78]:
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"Есть ли уроки": string  // вычлени из сообщения учителя информацию, будут ли завтра уроки или нет.                                          Если да, то пиши true, иначе false.
	"Количество уроков": string  // тут напиши, сколько именно будет уроков?.\ 
                                                     Если их не будет, значит пиши 0.
	"Необходимо_денег": string  // в данную переменную передай список всех указанных в                                                    тексте денежных сумм, если он есть.
}
```


In [79]:
review_template_2 = """\
Следуя нижеуказанным инструкциям, выдели информацию:\

Есть ли уроки: вычлени из сообщения учителя информацию, будут ли завтра уроки или нет.\
Если да, то пиши true, иначе false.\

Количество уроков: тут напиши, сколько именно будет уроков? Если их не будет, значит пиши 0.\

Необходимо_денег: в данную переменную передай список всех указанных в тексте денежных сумм, если он есть.\

Format the output as JSON with the following keys:
Есть ли уроки
Количество уроков
Необходимо денег

text: {text}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template=review_template_2)

messages = prompt.format_messages(text=teacher_message, 
                                format_instructions=format_instructions)

In [80]:
print(messages[0].content)

Следуя нижеуказанным инструкциям, выдели информацию:
Есть ли уроки: вычлени из сообщения учителя информацию, будут ли завтра уроки или нет.Если да, то пиши true, иначе false.
Количество уроков: тут напиши, сколько именно будет уроков? Если их не будет, значит пиши 0.
Необходимо_денег: в данную переменную передай список всех указанных в тексте денежных сумм, если он есть.
Format the output as JSON with the following keys:
Есть ли уроки
Количество уроков
Необходимо денег

text: 
Здравствуйте, ученики! Завтра в нашей школе большой праздник - столетие школы.Поэтому уроки на завтра отменяются!. Также напомните родителям, что в честь этого онидолжны сдать по две с половиной тысячи рублей на подарок директору! И ещё три тысячи мне на цветы!Люблю Вас!


The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"Есть ли уроки": string  // вычлени из сообщения учителя информацию, будут ли завтра уроки или н

In [81]:
response = get_completion(messages)

{
	"Есть ли уроки": "true",
	"Количество уроков": "0",
	"Необходимо_денег": "2500,3000"
}


In [84]:
output_dict = output_parser.parse(response)

In [85]:
output_dict

{'Есть ли уроки': 'true',
 'Количество уроков': '0',
 'Необходимо_денег': '2500,3000'}

In [86]:
type(output_dict)

dict

In [87]:
output_dict.get('Необходимо_денег')

'2500,3000'